In [1]:
#import tweepy
#import twint
#import nltk
#from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer
#from nltk.stem import Cistem
#from nltk.stem.snowball import GermanStemmer
#from nltk.tokenize import TweetTokenizer
#import re
#import regex as re
#import emoji
#import datetime
#import glob
import gensim as gs
import logging

import io
import os
import random
import string
#import unicodedata
#import time
import sys
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#from wordcloud import WordCloud
#import seaborn as sns
#from pprint import pprint

#import tensorflow as tf
#import tensorflow_datasets as tfds
#from tensorflow.keras import Model
#from tensorflow.keras import Sequential
#from tensorflow.keras.initializers import Constant
#from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
#from tensorflow.keras import layers

#import tqdm

# Workaround for Jupyter's "This event loop is already running" exception
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Check for GPU availability
#if tf.test.gpu_device_name():
#    print(tf.test.gpu_device_name())
#    print("GPU available.")
#else:
#    print("GPU not available.")
    
# Print numpy version
print(np.__version__) # Gensim needs a numpy version of >1.20 to work. Therefore we need two different environments...

1.20.2


In [3]:
handle="gecko203"
#handle="FiatPecunia"
# Read the tweets dataset
df = pd.read_csv('{}/preproc/tweets.csv'.format(handle))
#df = df['tweet']
df.head()

,bin,tweet,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,video
0,1,soh kar landerspiel portugal schenk bekomm gon...,False,0.0,0.0,0.486728,[],[],0
1,1,schad herrlich duf ruberbring pempelfor friedh...,True,0.0,1.0,0.570750,[],[],1
2,1,farb,True,0.0,0.0,0.557317,[],[],1
3,1,forum nrw aussengastronomie kaffee,True,0.0,0.0,0.519974,[],[],1
4,1,wett rheinla meer norweg frisch luf denk,True,1.0,1.0,0.602989,[],[],1


In [4]:
# Gensim needs an iterator yielding a list of tokens for each tweet.

for index, row in df.iterrows():
    print(row['tweet'].split())
    break

['soh', 'kar', 'landerspiel', 'portugal', 'schenk', 'bekomm', 'gonn', 'herz']


In [5]:
class Corpus:
    """ path must point to a CSV file with columns: rating,tweet,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,video """
    def __init__(self, path):
        #self.df = pd.read_csv(path,
        #             encoding='utf-8',
        #             dtype={'rating': float,
        #                    'tweet': 'string',
        #                    'photos': 'bool',
        #                    'replies_count': int,
        #                    'retweets_count': int,
        #                    'likes_count': int,
        #                    'hashtags': object,
        #                    'cashtags': object,
        #                    'video': int})
        self.df = pd.read_csv(path,
                     encoding='utf-8',
                     dtype={'rating': float,
                            'tweet': 'string',
                            'photos': 'bool',
                            'replies_count': float,
                            'retweets_count': float,
                            'likes_count': float,
                            'hashtags': object,
                            'cashtags': object,
                            'video': int})
        
        #print(self.df.loc[self.df['tweet'].isnull()])
        # Remove empty tweets
        self.df = self.df.dropna(subset=['tweet'])
    
    def __iter__(self):
        for index, row in self.df.iterrows():
            yield row['tweet'].split()

In [6]:
embedding_dim = 256 #128
seed = 42
epochs = 5 #2 #100 #5

corpus = Corpus('{}/preproc/tweets.csv'.format(handle))

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = gs.models.Word2Vec(sentences=corpus,
                           min_count=5,
                           sg=1,
                           seed=seed,
                           vector_size=embedding_dim,
                           workers=8,
                           epochs=epochs
                          )

2021-07-18 15:13:50,668 : INFO : collecting all words and their counts
2021-07-18 15:13:51,082 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-07-18 15:13:51,790 : INFO : PROGRESS: at sentence #10000, processed 76764 words, keeping 21454 word types
2021-07-18 15:13:52,417 : INFO : PROGRESS: at sentence #20000, processed 138534 words, keeping 31669 word types
2021-07-18 15:13:53,046 : INFO : PROGRESS: at sentence #30000, processed 214146 words, keeping 43064 word types
2021-07-18 15:13:53,684 : INFO : PROGRESS: at sentence #40000, processed 305201 words, keeping 53192 word types
2021-07-18 15:13:54,321 : INFO : PROGRESS: at sentence #50000, processed 374904 words, keeping 61236 word types
2021-07-18 15:13:54,959 : INFO : PROGRESS: at sentence #60000, processed 444707 words, keeping 68016 word types
2021-07-18 15:13:55,589 : INFO : PROGRESS: at sentence #70000, processed 516394 words, keeping 74319 word types
2021-07-18 15:13:56,220 : INFO : PROGRESS: at s

In [7]:
model.save('{}/preproc/gensim-w2v-model.gs'.format(handle))

2021-07-18 16:54:14,105 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'gecko203/preproc/gensim-w2v-model.gs', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-07-18T16:54:14.105747', 'gensim': '4.0.1', 'python': '3.9.5 (default, Jun  4 2021, 12:28:51) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.97-gentoo-x86_64-Intel-R-_Core-TM-_i7-4770K_CPU_@_3.50GHz-with-glibc2.33', 'event': 'saving'}
2021-07-18 16:54:14,106 : INFO : storing np array 'vectors' to gecko203/preproc/gensim-w2v-model.gs.wv.vectors.npy
2021-07-18 16:54:14,188 : INFO : storing np array 'syn1neg' to gecko203/preproc/gensim-w2v-model.gs.syn1neg.npy
2021-07-18 16:54:14,256 : INFO : not storing attribute cum_table
2021-07-18 16:54:14,303 : INFO : saved gecko203/preproc/gensim-w2v-model.gs


In [8]:
#model.wv.get_keras_embedding(train_embeddings=False)
model.wv.save('{}/preproc/gensim_w2v_vectors.gs'.format(handle))

2021-07-18 16:54:20,589 : INFO : KeyedVectors lifecycle event {'fname_or_handle': 'gecko203/preproc/gensim_w2v_vectors.gs', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-07-18T16:54:20.589484', 'gensim': '4.0.1', 'python': '3.9.5 (default, Jun  4 2021, 12:28:51) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.97-gentoo-x86_64-Intel-R-_Core-TM-_i7-4770K_CPU_@_3.50GHz-with-glibc2.33', 'event': 'saving'}
2021-07-18 16:54:20,590 : INFO : storing np array 'vectors' to gecko203/preproc/gensim_w2v_vectors.gs.vectors.npy
2021-07-18 16:54:20,701 : INFO : saved gecko203/preproc/gensim_w2v_vectors.gs


In [9]:
# For some weird reason, gensim can't decode its own file...

#model = gs.models.KeyedVectors.load_word2vec_format('preproc/gensim-w2v-model.gs', binary=False)

2021-07-11 18:11:50,250 : INFO : loading projection weights from preproc/gensim-w2v-model.gs


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [9]:
# So we have to save our own dataframe...

#print(model.wv[0])
#print(model.wv.index_to_key[0])
#print(model.wv.key_to_index['mal'])

vocab_size = len(model.wv.key_to_index)
tmp = {}

for i in range(vocab_size):
    word = model.wv.index_to_key[i]
    vec  = model.wv[i]
    
    tmp[word] = vec
    #newDF = newDF.append({word: vec}, ignore_index=True)

newDF = pd.DataFrame.from_dict({'word': [*tmp], 'vec': [*tmp.values()]})
#newDF = newDF.append(tmp, ignore_index=True)
#for i in range(vocab_size):
#    vec = model.wv[i]
#    if(vec is not None):
#        mat[i] = vec

#mat

newDF.head()

newDF.to_pickle('{}/preproc/gensim_w2v_dict.pkl'.format(handle))

In [10]:
for index, word in enumerate(model.wv.index_to_key):
    if index==10:
        break
    print(f"word #{index}/{len(model.wv.index_to_key)} is {word}")

word #0/135864 is mensch
word #1/135864 is amp
word #2/135864 is deutschla
word #3/135864 is frag
word #4/135864 is schon
word #5/135864 is deutsch
word #6/135864 is lieb
word #7/135864 is leb
word #8/135864 is seh
word #9/135864 is rech


In [11]:
try:
    vec_cameroon = model.wv['cameroon']
except KeyError:
        print("'cameroon' does not appear in this model")

'cameroon' does not appear in this model


In [15]:
pairs = [
    ('frau', 'frau'),
    ('frau', 'deutsch'),
    #('frau', 'hat'),
    ('frau', 'mann'),
    ('frau', 'mensch'),
    ('frau', 'eigentlich'),
    ('frau', 'zahnarz'),
    ('mann', 'zahnarz')
    #('mann', 'zahnarz')
]

for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, model.wv.similarity(w1, w2)))

'frau'	'frau'	1.00
'frau'	'deutsch'	0.16
'frau'	'mann'	0.69
'frau'	'mensch'	0.41
'frau'	'eigentlich'	0.16
'frau'	'zahnarz'	0.30
'mann'	'zahnarz'	0.29


In [21]:
print(model.wv.most_similar(positive=['frau', 'deutsch', 'einfach'], topn=10))

[('nichtsie', 0.6844909191131592), ('hochgearbei', 0.6753578186035156), ('frauenprobl', 0.6687976717948914), ('wegzulass', 0.6685789823532104), ('frauenkorp', 0.6660897135734558), ('intelligenzverscho', 0.6619774103164673), ('stutenbissigkei', 0.6585907936096191), ('zweitsprach', 0.6577056050300598), ('heiratsmark', 0.6572775840759277), ('hijabi', 0.6558840274810791)]


In [22]:
print(model.wv.most_similar(positive=['frau', 'mann', 'einfach'], topn=10))

[('frauenspor', 0.6991466879844666), ('mannerwel', 0.6986041069030762), ('sexobjek', 0.687872052192688), ('frauenkorp', 0.6873087286949158), ('mannerkleidung', 0.68612140417099), ('emanz', 0.6826204657554626), ('cismann', 0.6824772357940674), ('cisfrau', 0.6809875965118408), ('frauenprobl', 0.6774058938026428), ('hijabi', 0.6747159361839294)]


In [23]:
print(model.wv.most_similar(positive=['deutschla', 'deutsch'], topn=10))

[('dt', 0.6778990626335144), ('hierzula', 0.6307876706123352), ('deutschl', 0.5662012100219727), ('dtl', 0.5636727809906006), ('dieselkri', 0.5619921088218689), ('dtld', 0.5577219724655151), ('sozialleistungsempfang', 0.557549238204956), ('hereingelass', 0.5561498403549194), ('ordnungsmach', 0.5553346872329712), ('mutterla', 0.554523766040802)]


In [24]:
print(model.wv.most_similar(positive=['frau'], topn=10))

[('mann', 0.6940785646438599), ('transmann', 0.6129763126373291), ('frauenspor', 0.6056303977966309), ('sexobjek', 0.6037400960922241), ('feministi', 0.6025887727737427), ('transidentifizier', 0.6015456318855286), ('fuhrungspositio', 0.5989641547203064), ('penistrag', 0.5965065956115723), ('vagina', 0.5944669842720032), ('muslima', 0.5929049253463745)]


In [25]:
print(model.wv.most_similar(positive=['zahnarz'], topn=10))

[('orthopad', 0.6607589721679688), ('hno', 0.6541509628295898), ('blutabnahm', 0.6457017660140991), ('hautarz', 0.6441730856895447), ('rontg', 0.6424638628959656), ('kinderarzti', 0.6416801810264587), ('ekg', 0.6408825516700745), ('zahnreinigung', 0.6341341137886047), ('narko', 0.6268246173858643), ('ohrenarz', 0.6266707181930542)]


In [69]:
print(model.wv.most_similar(positive=['end'], topn=10))

[('beginn', 0.5218863487243652), ('beend', 0.5210116505622864), ('unvermeidlich', 0.48349034786224365), ('zahneklapp', 0.47610336542129517), ('nachwirk', 0.47291624546051025), ('allerspa', 0.4688767194747925), ('schlussak', 0.4680744409561157), ('endstatio', 0.46713894605636597), ('zwanzigerjahr', 0.4666043221950531), ('bacho', 0.46534284949302673)]


In [55]:
print(model.wv.most_similar(positive=['afd'], topn=10))

[('npd', 0.7239674925804138), ('hockeflugel', 0.7139185667037964), ('hockeafd', 0.7100300788879395), ('rechtsauss', 0.6934319138526917), ('volkischnationalistisch', 0.6930787563323975), ('unvereinbarkeitsbeschluss', 0.6846357583999634), ('leihstimm', 0.6803767085075378), ('sednachfolgepartei', 0.6752737164497375), ('harderkuhnel', 0.6730069518089294), ('nazipartei', 0.6728569865226746)]


In [58]:
print(model.wv.most_similar(positive=['cdu'], topn=10))

[('spd', 0.742838442325592), ('unio', 0.7420960664749146), ('unionspartei', 0.7076205611228943), ('cdubasi', 0.706127941608429), ('cduanhang', 0.6934468150138855), ('cduspitz', 0.6898589134216309), ('cducsu', 0.6809813380241394), ('csu', 0.6760632395744324), ('merkelcdu', 0.6737340092658997), ('wahlerauftrag', 0.671769380569458)]


In [59]:
print(model.wv.most_similar(positive=['csu'], topn=10))

[('cdu', 0.6760631799697876), ('regionalpartei', 0.6412636637687683), ('soder', 0.6325041055679321), ('dildoking', 0.6226835250854492), ('unio', 0.6220276951789856), ('spd', 0.6150292158126831), ('soderfa', 0.6105825901031494), ('cduspitz', 0.6097331643104553), ('plenk', 0.6036728024482727), ('csul', 0.6016107201576233)]


In [57]:
print(model.wv.most_similar(positive=['fdp'], topn=10))

[('spd', 0.6861640810966492), ('odp', 0.6713709235191345), ('lsa', 0.6647940874099731), ('leihstimm', 0.6515626907348633), ('jamaikakoalitio', 0.6503943204879761), ('umfallerpartei', 0.6494604349136353), ('koalitionsoptio', 0.6471983790397644), ('cdu', 0.6413511633872986), ('lindnerfdp', 0.6411862969398499), ('klimali', 0.6408944129943848)]


In [61]:
print(model.wv.most_similar(positive=['lkr'], topn=10))

[('vogtlandkrei', 0.6850035190582275), ('rheinberg', 0.6624794602394104), ('schmalkaldenmeining', 0.6567688584327698), ('mittelsach', 0.6467105150222778), ('direktkandidatur', 0.6433483362197876), ('reserveli', 0.6421079635620117), ('stormar', 0.641644299030304), ('holstenhall', 0.640998125076294), ('fraktionsstark', 0.6382033228874207), ('badenwurtemberg', 0.6370260119438171)]


In [60]:
print(model.wv.most_similar(positive=['gru'], topn=10))

[('grunro', 0.6162166595458984), ('verbotspartei', 0.6038878560066223), ('okopartei', 0.5963961482048035), ('gruninn', 0.5958684086799622), ('melonenpartei', 0.5911903977394104), ('fundi', 0.5856148600578308), ('angestrich', 0.5855979919433594), ('wunschpar', 0.5845481753349304), ('khmer', 0.581237256526947), ('okofaschi', 0.5809796452522278)]


In [62]:
print(model.wv.most_similar(positive=['spd'], topn=10))

[('cdu', 0.7428383231163025), ('unionspartei', 0.6882352232933044), ('fdp', 0.6861640810966492), ('nowabo', 0.6762338876724243), ('groko', 0.6740304827690125), ('keniakoalitio', 0.6716673374176025), ('koalitionsoptio', 0.6675721406936646), ('spdfuhrung', 0.6670170426368713), ('sednachfolgepartei', 0.6661092042922974), ('ampelkoalitio', 0.6628670692443848)]


In [63]:
print(model.wv.most_similar(positive=['dielink'], topn=10))

[('regierungsoptio', 0.8302925229072571), ('merkelpartei', 0.8232835531234741), ('linksbundni', 0.8183922171592712), ('wahlerauftrag', 0.8178619742393494), ('koaltio', 0.8168253302574158), ('exsed', 0.8125370740890503), ('regierungskonstellatio', 0.8124169111251831), ('mauermord', 0.8116545677185059), ('lieblingspartei', 0.8099782466888428), ('randpartei', 0.809731125831604)]


In [28]:
print(model.wv.most_similar(positive=['ezb'], topn=10))

[('zentralbank', 0.7351909875869751), ('ldpolitik', 0.7058497071266174), ('anleihekauf', 0.686378538608551), ('notenbank', 0.6733399033546448), ('zinspolitik', 0.6673334836959839), ('anleihenkauf', 0.6671331524848938), ('inflationsziel', 0.651358962059021), ('fed', 0.6488369703292847), ('eurozo', 0.6427165269851685), ('staatsanleih', 0.6344977617263794)]


In [47]:
print(model.wv.most_similar(positive=['ezb', 'lagard', 'euro'], topn=10))

[('anleihekauf', 0.7404888272285461), ('zinspolitik', 0.7400996685028076), ('anleihekaufprogramm', 0.7065806984901428), ('grexi', 0.7009917497634888), ('aufbaupla', 0.6952239871025085), ('vertragswidrig', 0.687871515750885), ('euzentralregim', 0.6877908110618591), ('wahrungsreform', 0.6876975297927856), ('griechenlandhilf', 0.6853057742118835), ('snb', 0.6836662888526917)]


In [54]:
print(model.wv.most_similar(positive=['shitcoi', 'euro'], topn=10))

[('umzutausch', 0.7948047518730164), ('eurojahr', 0.7848811745643616), ('euromona', 0.7765657305717468), ('dividendenausschuttung', 0.7746708989143372), ('mewirtschaf', 0.7744250297546387), ('reingewinn', 0.7738386988639832), ('fremdwahrung', 0.7736440300941467), ('leihgebuhr', 0.7702710032463074), ('munzgeld', 0.7702675461769104), ('eingesack', 0.7697898745536804)]


In [67]:
print(model.wv.most_similar(positive=['gold'], topn=10))

[('silb', 0.648666262626648), ('retriev', 0.5614104866981506), ('yamana', 0.5488790273666382), ('troilu', 0.545151948928833), ('barrick', 0.5450142025947571), ('edelmetall', 0.5359918475151062), ('sibany', 0.5355125665664673), ('newmo', 0.5328352451324463), ('stillwa', 0.5319861173629761), ('goldanleg', 0.5319448709487915)]


In [43]:
print(model.wv.doesnt_match(['ezb', 'rechtigkei', 'zukunf', 'stabilita']))

ezb


In [26]:
print(model.wv.doesnt_match(['frau', 'mann', 'land', 'einfach', 'lass', 'kind']))

einfach


In [27]:
print(model.wv.doesnt_match(['mal', 'einfach', 'land', 'deutschla', 'lass', 'kind']))

mal
